# set up

In [1]:
import pandas as pd
import re
from collections import Counter

In [2]:
path  = "./ws.xlsx"

In [3]:
df_dict = pd.read_excel(path, sheet_name=None )

# exploring

In [192]:
df_dict.keys()

dict_keys(['2023', '2020', '2017', '2012', '2007', '2002'])

In [6]:
df2023 = df_dict['2023']

In [194]:
df2023.head()

,"Year Ended December 31(Dollars and Shares in Millions, Except Per Share Data)",2023,2022,2021,2023v 2022,2022v 2021
0,Net interest income,17396.0,14728.0,12494.0,18.1%,17.9%
1,Taxable-equivalent adjustment^{(a)},131.0,118.0,106.0,11.0,11.3
2,Net interest income (taxable-equivalent basis)...,17527.0,14846.0,12600.0,18.1,17.8
3,Noninterest income,10617.0,9456.0,10227.0,12.3,(7.5)
4,Total net revenue,28144.0,24302.0,22827.0,15.8,6.5


In [195]:
ser = df2023.iloc[:, 0]
ser.name
df2023.columns[0]


'Year Ended December 31(Dollars and Shares in Millions, Except Per Share Data)'

# beginning implementation 

In [7]:
functions = [
    lambda s : re.sub(r"\^\{\(.*\)\}", "", s),
    lambda s : re.sub(r"\(.*\)", "", s),
    lambda s : re.split(r"\n", s)[-1],
    lambda s: s.lower()
]
def apply_functions(s): 
    for f in functions:
        s = f(s)
    return s

In [5]:
clean_df = df2023.copy()
clean_df.iloc[:, 0] = clean_df.iloc[:,0].apply(apply_functions)


NameError: name 'df2023' is not defined

In [8]:
def clean_df_dict(df_dict: dict[str, pd.DataFrame]) -> dict[str, pd.DataFrame]: 
    clean_dict = dict()
    for key,df in df_dict.items():
        clean_df = df.copy()
        clean_df.iloc[:, 0] = clean_df.iloc[:,0].apply(apply_functions)
        clean_dict[key] = clean_df
    return clean_dict

clean_dict = clean_df_dict(df_dict)
clean_dict['2023'].head()


,"Year Ended December 31(Dollars and Shares in Millions, Except Per Share Data)",2023,2022,2021,2023v 2022,2022v 2021
0,net interest income,17396.0,14728.0,12494.0,18.1%,17.9%
1,taxable-equivalent adjustment,131.0,118.0,106.0,11.0,11.3
2,net interest income,17527.0,14846.0,12600.0,18.1,17.8
3,noninterest income,10617.0,9456.0,10227.0,12.3,(7.5)
4,total net revenue,28144.0,24302.0,22827.0,15.8,6.5


In [9]:
def get_columns(df_dict, label_column=0, start_row=0): 
    """    
    returns a pair of set of shared columns and a dict of name:set of not shared columns
    expects a dictionary of 'name':'df' where each df has the same starting column and row 

    Args:
        df_dict (dict[str:df]): mapping from name to dataframe. All dataframs are expected to be of similar structure
        label_column (int, optional): the column of labels. Defaults to 0.
        start_row (int, optional): the starting of row. Defaults to 0.
    """
    row_labels = None
    for df in df_dict.values():
        df_cleaned = df.iloc[start_row:, label_column].apply(apply_functions)
        if not row_labels:
            row_labels = set(df_cleaned)
        row_labels &= set(df_cleaned)
    uniques = dict()
    for name, df in df_dict.items():
        df_cleaned = df.iloc[start_row:, label_column].apply(apply_functions)
        uniques[name] = set(df_cleaned) - row_labels

    return row_labels, uniques
row_labels, uniques = get_columns(clean_dict)


In [10]:
def get_columns_counter(df_dict, label_column=0, start_row=0): 
    """    
    returns a counter of column names
    expects a dictionary of 'name':'df' where each df has the same starting column and row 

    Args:
        df_dict (dict[str:df]): mapping from name to dataframe. All dataframs are expected to be of similar structure
        label_column (int, optional): the column of labels. Defaults to 0.
        start_row (int, optional): the starting of row. Defaults to 0.
    """
    row_counter = Counter()
    for df in df_dict.values():
        df_cleaned = df.iloc[start_row:, label_column].apply(apply_functions)
        print(set(df_cleaned))
        break
        row_counter += Counter(df_cleaned)
    return row_counter

row_counter = get_columns_counter(clean_dict)
row_counter


{'noninterest expense', 'net income', 'average balances', 'implementation of the current expected credit losses methodology', 'net interest income ', 'total risk-based capital', 'dividends declared per share', 'allowance for credit losses', 'leverage', 'capital ratios', 'income taxes and taxable-equivalent adjustment', 'average diluted common shares outstanding', 'market value per share', 'tangible common equity to risk-weighted assets', 'total leverage exposure', 'efficiency ratio', 'taxable-equivalent adjustment', 'tier 1 capital', 'tangible common equity to tangible assets', 'period end balances', 'return on average assets', 'diluted earnings per share', 'noninterest income', 'investment securities', 'earnings per share', 'net income attributable to u.s. bancorp', 'net interest income', 'return on average common equity', 'provision for credit losses', 'net income applicable to u.s. bancorp common shareholders', 'per common share', 'earning assets', 'deposits', "total u.s. bancorp sh

Counter()

In [ ]:
row_labels

In [ ]:
uniques

# using pds

In [19]:
from typing import List

In [28]:
clean_dict = clean_df_dict(df_dict)
clean_dict['2023'].iloc[:, :2]
small_dfs: List[pd.DataFrame] = [df.iloc[:,:2].set_index(df.columns[0]) for df in clean_dict.values()]
joined = small_dfs[0].join(small_dfs[1:], how='inner')
joined


,2023,2020,2017,2012,2007,2002
"Year Ended December 31(Dollars and Shares in Millions, Except Per Share Data)",,,,,,
noninterest expense,18873.00,13369.00,12945.00,"10,456","6,862",5932.50
provision for credit losses,2275.00,3806.00,1390.00,"1,882",792,1349.00
net income,5458.00,4985.00,6253.00,"5,490","$4,324",3289.20
per common share,NaN,NaN,NaN,NaN,NaN,NaN
earnings per share,3.27,3.06,3.53,$2.85,$2.46,1.72
...,...,...,...,...,...,...
deposits,512312.00,429770.00,347215.00,"249,183","131,445",105124.00
deposits,512312.00,429770.00,347215.00,"249,183","131,445",115534.00
tier 1 capital,11.50,11.30,10.80,10.8%,8.3%,7.80
